In [1]:
# -*- coding: utf-8 -*-
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6'
# ================= Monkey‐patch для PIL и numpy ===================
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

from PIL import Image
if not hasattr(Image, 'LINEAR'):
    from PIL.Image import Resampling
    Image.NEAREST  = Resampling.NEAREST
    Image.BILINEAR = Resampling.BILINEAR
    Image.BICUBIC  = Resampling.BICUBIC
    Image.LANCZOS  = Resampling.LANCZOS
    Image.BOX      = Resampling.BOX
    Image.HAMMING  = Resampling.HAMMING
    Image.LINEAR   = Resampling.BILINEAR

# ================= Импорты ===================
import os, random, json, pandas as pd, cv2, matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter


  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-req-build-5pc5zt84
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-5pc5zt84
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.1 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6424848 sha256=dc0e77d13eefe0954703a47d8f86503d642ec1e6dae6aaa27e1d607dd0d0

/tmp/ipykernel_86/2732540068.py:5: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):
2025-05-15 18:31:01.415615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747333861.437587      86 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747333861.444272      86 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor, HookBase, EvalHook
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.datasets import load_coco_json
from detectron2.data.dataset_mapper import DatasetMapper
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.logger import setup_logger

import math

In [3]:
# ================= Параметры пути ===================
DATA_ROOT   = '/kaggle/working/wheels5k-coco/train'
IMG_DIR     = '/kaggle/input/wheels5k-coco/train'
DS_DIR      = os.path.join('/kaggle/working/wheels5k-coco', 'datasets_filtered')  # JSON после remap category_id->0
OUT_ROOT    = os.path.join('/kaggle/working/', 'output')
os.makedirs(OUT_ROOT, exist_ok=True)

# TensorBoard и CSV
tb_writer = SummaryWriter(log_dir=os.path.join(OUT_ROOT, 'tensorboard'))
CSV_PATH  = os.path.join(OUT_ROOT, 'metrics.csv')



In [4]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
from pycocotools import mask as mask_utils

coco_path = '/kaggle/input/wheels5k-coco/train/_annotations.coco.json'
images_dir = '/kaggle/input/wheels5k-coco/train'

with open(coco_path, 'r') as f:
    coco = json.load(f)

def filter_by_tag(imgs, tag):
    return [img for img in imgs
            if 'extra' in img 
            and 'user_tags' in img['extra'] 
            and tag in img['extra']['user_tags']]

start_imgs = filter_by_tag(coco['images'], 'start')
end_imgs   = filter_by_tag(coco['images'], 'end')

def split(imgs):
    tr, v = train_test_split(imgs, test_size=0.2, random_state=42)
    v, te = train_test_split(v, test_size=0.5, random_state=42)
    return tr, v, te

train_start, val_start, test_start = split(start_imgs)
train_end,   val_end,   test_end   = split(end_imgs)

test_imgs = test_start + test_end

def filter_anns(imgs):
    ids = set(i['id'] for i in imgs)
    return [a for a in coco['annotations'] if a['image_id'] in ids]

splits = {
    'test': (test_imgs, filter_anns(test_imgs)),
    'val_start':   (val_start,   filter_anns(val_start)),
    'val_end':   (val_end,   filter_anns(val_end)),
    'train_start': (train_start, filter_anns(train_start)),
    'train_end':   (train_end,   filter_anns(train_end))
}

os.makedirs('/kaggle/working/wheels5k-coco/datasets1', exist_ok=True)
for name, (imgs, anns) in splits.items():
    out = {'images': imgs, 'annotations': anns, 'categories': coco['categories']}
    with open(f'/kaggle/working/wheels5k-coco/datasets1/{name}.json','w') as f:
        json.dump(out, f)


In [5]:
import json, os

def filter_and_remap_coco(input_json, output_json, target_cat_id=6):
    data = json.load(open(input_json))
    # Оставляем только нужные category и ann
    new_anns = []
    img_ids = set()
    for ann in data['annotations']:
        if ann['category_id'] == target_cat_id:
            ann['category_id'] = 0   # Remap to 0
            new_anns.append(ann)
            img_ids.add(ann['image_id'])
    # Фильтруем изображения
    new_imgs = [img for img in data['images'] if img['id'] in img_ids]
    # Новое описание категории
    new_cats = [{"id": 0, "name": "wheel"}]
    # Сохраняем
    out = {
        "images": new_imgs,
        "annotations": new_anns,
        "categories": new_cats
    }
    json.dump(out, open(output_json, "w"), indent=2)
    print(f"Filtered {len(data['annotations'])}->{len(new_anns)} annotations, images {len(data['images'])}->{len(new_imgs)}")

# Примените к вашим файлам
os.makedirs("/kaggle/working/wheels5k-coco/datasets_filtered", exist_ok=True)
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/train_start.json", "/kaggle/working/wheels5k-coco/datasets_filtered/start_train.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/val_start.json",   "/kaggle/working/wheels5k-coco/datasets_filtered/start_val.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/train_end.json", "/kaggle/working/wheels5k-coco/datasets_filtered/end_train.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/val_end.json",   "/kaggle/working/wheels5k-coco/datasets_filtered/end_val.json")

Filtered 5365->3416 annotations, images 1728->1437
Filtered 648->432 annotations, images 216->178
Filtered 4980->3293 annotations, images 1828->1506
Filtered 619->426 annotations, images 228->193


In [6]:
# ================= Регистрация датасетов ===================
def register_split(name, json_file):
    DatasetCatalog.register(name, lambda: load_coco_json(json_file, IMG_DIR))
    MetadataCatalog.get(name).set(thing_classes=['wheel'], evaluator_type='coco')

# Сплиты train/val для start и end (используют один и тот же remapped JSON)

#register_split('t_end',   os.path.join(DS_DIR, 'end_train.json'))
#register_split('v_end',   os.path.join(DS_DIR, 'end_val.json'))
register_split('_t_start', os.path.join(DS_DIR, 'start_train.json'))
register_split('_v_start', os.path.join(DS_DIR, 'start_val.json'))


In [7]:
# ================= Функция генерации конфига ===================
def make_cfg(train_ds, val_ds, output_dir,
             max_iter=16160, base_lr=1e-3, batch_size=4):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    # Датасеты
    cfg.DATASETS.TRAIN        = (train_ds,)
    cfg.DATASETS.TEST         = (val_ds,)
    # Dataloader
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.SOLVER.IMS_PER_BATCH   = batch_size
    # LR-стратегия
    cfg.SOLVER.BASE_LR         = base_lr
    cfg.SOLVER.WEIGHT_DECAY    = 1e-4
    cfg.SOLVER.WARMUP_ITERS    = 800
    cfg.SOLVER.WARMUP_METHOD   = "linear"
    cfg.SOLVER.MAX_ITER        = max_iter
    cfg.SOLVER.STEPS           = (int(max_iter * 0.6), int(max_iter * 0.9))
    cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
    cfg.TEST.EVAL_PERIOD       = 377  # валидация каждые 500 итераций
    

    # ROI Heads
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 48
    cfg.INPUT.MAX_SIZE_TRAIN = 1024
    cfg.INPUT.MAX_SIZE_TEST  = 1024
    # Вывод
    cfg.OUTPUT_DIR = output_dir
    os.makedirs(output_dir, exist_ok=True)

    cfg.MODEL.DEVICE = "cuda"
    return cfg



In [8]:
class BestCheckpointer(HookBase):
    def __init__(self, checkpointer, eval_period, save_name="model_best"):
        self.checkpointer = checkpointer
        self.eval_period  = eval_period
        self.best_metric  = -float("inf")
        self.save_name    = save_name

    def after_step(self):
        it = self.trainer.iter
        if it == 0 or (it + 1) % self.eval_period != 0:
            return
        latest = self.trainer.storage.latest()
        if "segm/AP50" not in latest:
            return
        current = latest["segm/AP50"][0]
        if current > self.best_metric:
            self.best_metric = current
            self.checkpointer.save(self.save_name)
            print(f"[BestCheckpointer] New best segm/AP50 = {current:.4f} at iter {it+1}")




In [9]:
import torch
import detectron2.utils.comm as comm
from detectron2.engine import HookBase
from detectron2.data import build_detection_test_loader
from detectron2.data.dataset_mapper import DatasetMapper

class ValidationLossHook(HookBase):
    def __init__(self, eval_period, trainer):
        """
        eval_period: сколько шагов между запуском валидации
        trainer:     экземпляр вашего MyTrainer
        """
        self._period  = eval_period
        self.trainer  = trainer
        # сборка val-loader, но с is_train=True, чтобы были gt_instances
        cfg = trainer.cfg
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[])
        self._data_loader = iter(
            build_detection_test_loader(cfg, cfg.DATASETS.TEST[0], mapper=mapper)
        )

    def _get_val_loss(self):
        # получаем следующий батч
        try:
            data = next(self._data_loader)
        except StopIteration:
            # если даталоадер кончился, пересоздаём
            cfg    = self.trainer.cfg
            mapper = DatasetMapper(cfg, is_train=True, augmentations=[])
            self._data_loader = iter(
                build_detection_test_loader(cfg, cfg.DATASETS.TEST[0], mapper=mapper)
            )
            data = next(self._data_loader)

        # вычисляем лосс
        with torch.no_grad():
            loss_dict = self.trainer.model(data)  # training mode → вернёт dict losses
            # суммируем и редуцируем по всем устройствам
            losses_reduced = comm.reduce_dict(loss_dict)
            total_val_loss = sum(v.item() for v in losses_reduced.values())
            # префикс 'val_' для всех скаляров
            loss_dict_reduced = {
                f"val_{k}": v.item() for k, v in losses_reduced.items()
            }

        return total_val_loss, loss_dict_reduced

    def after_step(self):
        next_iter = self.trainer.iter + 1
        # запускаем каждые eval_period шагов и на последнем
        if next_iter % self._period != 0 and next_iter != self.trainer.max_iter:
            return
        total_val_loss, loss_dict_reduced = self._get_val_loss()
        if comm.is_main_process():
            # записываем в storage, чтобы их писал Tensorboard/X
            self.trainer.storage.put_scalar("total_val_loss", total_val_loss)
            self.trainer.storage.put_scalars(**loss_dict_reduced)

In [10]:
# ================= Custom Trainer ===================
from detectron2.engine import hooks as d2_hooks
from detectron2.data import build_detection_train_loader

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        # 1) Считаем, во сколько пикселей надо уменьшить,
        #    взяв оригинальный диапазон MIN_SIZE_TRAIN и разделив на 2.5
        #min_size   = cfg.INPUT.MIN_SIZE_TRAIN
        #max_size   = cfg.INPUT.MAX_SIZE_TRAIN
        #scale      = 2
        #new_min    = max(1, math.floor(min_size/scale))
        #new_max    = max(1, math.floor(max_size/scale))

        # 2) Формируем список аугментаций, начиная с простого ресайза
        aug_list = [
        #    T.ResizeShortestEdge(new_min, new_max, sample_style="choice"),
        ]
        mapper = DatasetMapper(cfg, is_train=True, augmentations=aug_list)
        return build_detection_train_loader(cfg, mapper=mapper)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        # ТО-ЖЕ самое для валидации / инференса
        #min_size = cfg.INPUT.MIN_SIZE_TEST
        #max_size = cfg.INPUT.MAX_SIZE_TEST
        #scale    = 2.5
        #new_min  = max(1, math.floor(min_size/scale))
        #new_max  = max(1, math.floor(max_size/scale))

        aug_list = [
        #    T.ResizeShortestEdge([new_min], new_max, sample_style="choice"),
        ]
        mapper = DatasetMapper(cfg, is_train=False, augmentations=aug_list)
        return build_detection_test_loader(cfg, dataset_name, mapper=mapper)
        
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        evaluator = COCOEvaluator(
        dataset_name,
        tasks=["bbox","segm"],
        distributed=False,
        output_dir=cfg.OUTPUT_DIR,
        )
        return evaluator

    @classmethod
    def build_optimizer(cls, cfg, model):
        lr = cfg.SOLVER.BASE_LR
        # Собираем все параметры backbone
        backbone_set = set(model.backbone.parameters())
        # Две группы: backbone и всё остальное
        backbone_params = []
        other_params    = []
        for p in model.parameters():
            if not p.requires_grad:
                continue
            if p in backbone_set:
                backbone_params.append(p)
            else:
                other_params.append(p)
        optimizer = torch.optim.SGD(
            [
              {"params": backbone_params, "lr": lr * 0.1},
              {"params": other_params,    "lr": lr}
            ],
            lr=lr, momentum=0.9, weight_decay=cfg.SOLVER.WEIGHT_DECAY
        )
        return optimizer

    def build_hooks(self):
        hooks_list = super().build_hooks()
        insertion_idx = len(hooks_list) - 1
        # вставляем наш ValidationLossHook
        val_hook = ValidationLossHook(
            eval_period=self.cfg.TEST.EVAL_PERIOD,
            trainer=self
        )
        hooks_list.insert(insertion_idx, val_hook)
        # Вставляем после EvalHook
        for idx, h in enumerate(hooks_list):
            if isinstance(h, EvalHook):
                checkpointer = DetectionCheckpointer(self.model, self.cfg.OUTPUT_DIR)
                best_cp = BestCheckpointer(checkpointer,
                                           eval_period=self.cfg.TEST.EVAL_PERIOD,
                                           save_name="model_best")
                hooks_list.insert(idx+1, best_cp)
                break
        return hooks_list


In [ ]:
# --- End split ---
setup_logger()

cfg_e = make_cfg("t_end", "v_end", os.path.join(OUT_ROOT, "end"), max_iter=18850)
trainer_e = MyTrainer(cfg_e)
trainer_e.resume_or_load(resume=True)
trainer_e.train()

[05/15 18:31:11 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/15 18:31:36 d2.utils.events]:  eta: 5:43:32  iter: 19  total_loss: 1.202  loss_cls: 0.19  loss_box_reg: 0.1137  loss_mask: 0.6872  loss_rpn_cls: 0.139  loss_rpn_loc: 0.02313  time: 1.0969  data_time: 0.0296  lr: 2.3378e-06  max_mem: 10504M
[05/15 18:31:58 d2.utils.events]:  eta: 5:43:31  iter: 39  total_loss: 1.255  loss_cls: 0.2134  loss_box_reg: 0.1705  loss_mask: 0.6772  loss_rpn_cls: 0.148  loss_rpn_loc: 0.02402  time: 1.1005  data_time: 0.0152  lr: 4.8242e-06  max_mem: 10505M
[05/15 18:32:20 d2.utils.events]:  eta: 5:43:57  iter: 59  total_loss: 1.268  loss_cls: 0.2169  loss_box_reg: 0.1892  loss_mask: 0.683  loss_rpn_cls: 0.1359  loss_rpn_loc: 0.02331  time: 1.1044  data_time: 0.0158  lr: 7.3106e-06  max_mem: 10505M
[05/15 18:32:42 d2.utils.events]:  eta: 5:43:52  iter: 79  total_loss: 1.173  loss_cls: 0.1947  loss_box_reg: 0.1729  loss_mask: 0.6876  loss_rpn_cls: 0.1241  loss_rpn_loc: 0.02051  time: 1.1056  data_time: 0.0157  lr: 9.797e-06  max_mem: 10505M
[05/15 18:33:05 d2

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
register_split('_t_start', os.path.join(DS_DIR, 'start_train.json'))
register_split('_v_start', os.path.join(DS_DIR, 'start_val.json'))

In [ ]:
# ================= Оценка и лог в CSV + TensorBoard ===================
def evaluate_and_log(trainer, cfg, split, tag):
    evaluator  = COCOEvaluator(split, cfg, False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, split)
    metrics    = inference_on_dataset(trainer.model, val_loader, evaluator)
    segm       = metrics.get("segm", {})
    row = {
        "tag":     tag,
        "AP50:95": segm.get("AP", 0),
        "AP50":    segm.get("AP50",0),
        "AP75":    segm.get("AP75",0)
    }
    # CSV
    df = pd.DataFrame([row])
    df.to_csv(CSV_PATH, mode='a', header=not os.path.exists(CSV_PATH), index=False)
    # TensorBoard
    for k,v in row.items():
        if k!="tag": tb_writer.add_scalar(f"val/{tag}/{k}", v)
    return metrics

metrics_s = evaluate_and_log(trainer_s, cfg_s, "v_start", "start")
metrics_e = evaluate_and_log(trainer_e, cfg_e, "v_end",   "end")

# ================= Инференс и визуализация 4 примеров ===================
predictor_s = DefaultPredictor(cfg_s)
predictor_e = DefaultPredictor(cfg_e)
meta_val    = MetadataCatalog.get("v_start")  # общий метадатасет
ds_val      = DatasetCatalog.get("v_start")

for d in random.sample(ds_val, 4):
    img = cv2.imread(os.path.join(IMG_DIR, d["file_name"]))
    tag = "start" if "start" in d.get("extra",{}).get("user_tags",[]) else "end"
    pred = (predictor_s if tag=="start" else predictor_e)(img)
    vis  = Visualizer(img[:,:,::-1], metadata=meta_val, scale=0.8)
    out  = vis.draw_instance_predictions(pred["instances"].to("cpu"))
    plt.figure(figsize=(8,6))
    plt.imshow(out.get_image()[:,:,::-1])
    plt.title(f"{tag} | {d['file_name']}")
    plt.axis("off")
    plt.show()

print("Пайплайн завершён! Метрики сохранены в", CSV_PATH)


In [ ]:
# trainer = ваш объект DefaultTrainer или его наследник
# cfg - ваш конфиг, cfg.OUTPUT_DIR указывает папку для чекпоинтов

checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)  # [1]
checkpointer.save("model_final")  # сохранит файл OUTPUT_DIR/model_final.pth  [1]

In [ ]:
# model - тот же состав модели, что и при обучении
checkpointer_ = DetectionCheckpointer(model, save_dir=(f'cfg.OUTPUT_DIR'+'/start')
checkpointer.load("model_best.pth")  # загрузит веса в model  [2]

In [ ]:
# ================= Main: тренировка start и end ===================
setup_logger()

# --- Start split ---
cfg_s = make_cfg("_t_start", "_v_start", os.path.join(OUT_ROOT, "start"), max_iter=16160)
trainer_s = MyTrainer(cfg_s)
trainer_s.resume_or_load(resume=False)
trainer_s.train()

# --- End split ---
cfg_e = make_cfg("_t_end", "_v_end", os.path.join(OUT_ROOT, "end"), max_iter=16160)
trainer_e = MyTrainer(cfg_e)
trainer_e.resume_or_load(resume=False)
trainer_e.train()

